# Euler Method for Solving Ordinary Differential Equations

## Introduction

The **Euler method** is the simplest and most fundamental numerical technique for solving initial value problems (IVPs) of ordinary differential equations (ODEs). Named after the Swiss mathematician Leonhard Euler (1707–1783), this method provides the foundation for understanding more sophisticated numerical integration schemes.

## Mathematical Foundation

### The Initial Value Problem

Consider a first-order ODE of the form:

$$\frac{dy}{dt} = f(t, y), \quad y(t_0) = y_0$$

where $f(t, y)$ is a known function, $t_0$ is the initial time, and $y_0$ is the initial condition.

### Derivation of Euler's Method

The method is derived from the Taylor series expansion of $y(t)$ about the point $t_n$:

$$y(t_{n+1}) = y(t_n) + h\frac{dy}{dt}\bigg|_{t_n} + \frac{h^2}{2}\frac{d^2y}{dt^2}\bigg|_{t_n} + O(h^3)$$

where $h = t_{n+1} - t_n$ is the step size.

Truncating after the first derivative term and using the differential equation, we obtain the **Euler formula**:

$$y_{n+1} = y_n + h \cdot f(t_n, y_n)$$

### Local and Global Truncation Error

The **local truncation error** (LTE) is $O(h^2)$, arising from the neglected terms in the Taylor expansion:

$$\text{LTE} = \frac{h^2}{2}y''(\xi), \quad \xi \in [t_n, t_{n+1}]$$

The **global truncation error** accumulates over $N = (t_f - t_0)/h$ steps, giving:

$$\text{GTE} = O(h)$$

Thus, Euler's method is a **first-order method**.

### Stability Considerations

For the test equation $\frac{dy}{dt} = \lambda y$ with $\lambda < 0$, the Euler method is stable when:

$$|1 + h\lambda| < 1$$

This defines the stability region in the complex plane.

## Implementation

We will implement the Euler method and apply it to several test cases to demonstrate its behavior, accuracy, and limitations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Set plotting style
plt.rcParams['figure.figsize'] = [12, 10]
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

In [ ]:
def euler_method(f, y0, t0, tf, h):
    """
    Solve an ODE using the Euler method.
    
    Parameters
    ----------
    f : callable
        Function f(t, y) defining the ODE dy/dt = f(t, y)
    y0 : float
        Initial condition y(t0) = y0
    t0 : float
        Initial time
    tf : float
        Final time
    h : float
        Step size
    
    Returns
    -------
    t : ndarray
        Array of time points
    y : ndarray
        Array of solution values
    """
    # Number of steps
    n_steps = int(np.ceil((tf - t0) / h))
    
    # Initialize arrays
    t = np.zeros(n_steps + 1)
    y = np.zeros(n_steps + 1)
    
    # Set initial conditions
    t[0] = t0
    y[0] = y0
    
    # Euler iteration
    for i in range(n_steps):
        y[i+1] = y[i] + h * f(t[i], y[i])
        t[i+1] = t[i] + h
    
    return t, y

## Example 1: Exponential Decay

Consider the radioactive decay equation:

$$\frac{dy}{dt} = -\lambda y, \quad y(0) = y_0$$

The analytical solution is:

$$y(t) = y_0 e^{-\lambda t}$$

This problem allows us to directly compare the numerical solution with the exact solution.

In [ ]:
# Define the ODE for exponential decay
lambda_decay = 0.5  # decay constant
f_decay = lambda t, y: -lambda_decay * y

# Analytical solution
y0_decay = 1.0
y_exact = lambda t: y0_decay * np.exp(-lambda_decay * t)

# Solve with different step sizes
t0, tf = 0.0, 10.0
step_sizes = [2.0, 1.0, 0.5, 0.1]

# Store solutions
solutions_decay = []
for h in step_sizes:
    t, y = euler_method(f_decay, y0_decay, t0, tf, h)
    solutions_decay.append((t, y, h))

# Compute exact solution for plotting
t_fine = np.linspace(t0, tf, 1000)
y_fine = y_exact(t_fine)

## Example 2: Logistic Growth

The logistic equation models population growth with carrying capacity:

$$\frac{dP}{dt} = rP\left(1 - \frac{P}{K}\right)$$

where $r$ is the growth rate and $K$ is the carrying capacity.

The analytical solution is:

$$P(t) = \frac{K P_0 e^{rt}}{K + P_0(e^{rt} - 1)}$$

In [ ]:
# Logistic growth parameters
r = 1.0  # growth rate
K = 100.0  # carrying capacity
P0 = 10.0  # initial population

# Define the ODE
f_logistic = lambda t, P: r * P * (1 - P/K)

# Analytical solution
P_exact = lambda t: (K * P0 * np.exp(r*t)) / (K + P0 * (np.exp(r*t) - 1))

# Solve with Euler method
t0, tf = 0.0, 10.0
h_logistic = 0.2
t_log, P_log = euler_method(f_logistic, P0, t0, tf, h_logistic)

# Fine solution for comparison
t_fine_log = np.linspace(t0, tf, 1000)
P_fine = P_exact(t_fine_log)

## Example 3: Simple Harmonic Oscillator

The simple harmonic oscillator is described by:

$$\frac{d^2x}{dt^2} = -\omega^2 x$$

This second-order ODE can be converted to a system of first-order ODEs:

$$\frac{dx}{dt} = v, \quad \frac{dv}{dt} = -\omega^2 x$$

The analytical solution is:

$$x(t) = A\cos(\omega t + \phi)$$

This example demonstrates the energy drift problem inherent in explicit Euler methods.

In [ ]:
def euler_system(f, y0, t0, tf, h):
    """
    Solve a system of ODEs using the Euler method.
    
    Parameters
    ----------
    f : callable
        Function f(t, y) returning array of derivatives
    y0 : array-like
        Initial conditions
    t0, tf : float
        Initial and final times
    h : float
        Step size
    
    Returns
    -------
    t : ndarray
        Time points
    y : ndarray
        Solution array (n_steps+1 x n_vars)
    """
    y0 = np.array(y0)
    n_steps = int(np.ceil((tf - t0) / h))
    n_vars = len(y0)
    
    t = np.zeros(n_steps + 1)
    y = np.zeros((n_steps + 1, n_vars))
    
    t[0] = t0
    y[0] = y0
    
    for i in range(n_steps):
        y[i+1] = y[i] + h * np.array(f(t[i], y[i]))
        t[i+1] = t[i] + h
    
    return t, y

# Harmonic oscillator parameters
omega = 2.0 * np.pi  # angular frequency (period = 1)

# System of ODEs: [x, v] -> [dx/dt, dv/dt]
f_harmonic = lambda t, y: [y[1], -omega**2 * y[0]]

# Initial conditions: x(0) = 1, v(0) = 0
y0_harm = [1.0, 0.0]

# Solve for multiple periods
t0, tf = 0.0, 5.0
h_harm = 0.01
t_harm, y_harm = euler_system(f_harmonic, y0_harm, t0, tf, h_harm)

# Exact solution
x_exact_harm = lambda t: np.cos(omega * t)
v_exact_harm = lambda t: -omega * np.sin(omega * t)

## Convergence Analysis

To verify that our implementation is correct and to demonstrate the first-order convergence of the Euler method, we compute the global error for decreasing step sizes.

In [ ]:
# Convergence study for exponential decay
step_sizes_conv = [0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625]
errors = []

for h in step_sizes_conv:
    t, y = euler_method(f_decay, y0_decay, 0.0, 5.0, h)
    # Global error at t = 5
    error = np.abs(y[-1] - y_exact(t[-1]))
    errors.append(error)

errors = np.array(errors)
step_sizes_conv = np.array(step_sizes_conv)

# Compute convergence rate
log_h = np.log(step_sizes_conv)
log_err = np.log(errors)
slope, intercept = np.polyfit(log_h, log_err, 1)
print(f"Convergence rate: {slope:.3f} (expected: 1.0)")

## Visualization

In [ ]:
# Create comprehensive figure
fig = plt.figure(figsize=(14, 12))
gs = GridSpec(3, 2, figure=fig, hspace=0.3, wspace=0.25)

# Plot 1: Exponential decay with different step sizes
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t_fine, y_fine, 'k-', linewidth=2, label='Exact')
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(solutions_decay)))
for (t, y, h), color in zip(solutions_decay, colors):
    ax1.plot(t, y, 'o-', color=color, markersize=4, 
             label=f'h = {h}', alpha=0.7)
ax1.set_xlabel('Time t')
ax1.set_ylabel('y(t)')
ax1.set_title('Exponential Decay: Effect of Step Size')
ax1.legend(loc='upper right')
ax1.set_xlim([0, 10])

# Plot 2: Convergence analysis
ax2 = fig.add_subplot(gs[0, 1])
ax2.loglog(step_sizes_conv, errors, 'bo-', markersize=8, linewidth=2, label='Computed error')
# Reference line for O(h)
h_ref = np.array([step_sizes_conv[0], step_sizes_conv[-1]])
err_ref = errors[0] * (h_ref / step_sizes_conv[0])
ax2.loglog(h_ref, err_ref, 'r--', linewidth=2, label=f'O(h), slope = {slope:.2f}')
ax2.set_xlabel('Step size h')
ax2.set_ylabel('Global error at t = 5')
ax2.set_title('Convergence Analysis')
ax2.legend()

# Plot 3: Logistic growth
ax3 = fig.add_subplot(gs[1, 0])
ax3.plot(t_fine_log, P_fine, 'k-', linewidth=2, label='Exact')
ax3.plot(t_log, P_log, 'ro-', markersize=4, alpha=0.7, label=f'Euler (h={h_logistic})')
ax3.axhline(y=K, color='gray', linestyle='--', alpha=0.5, label=f'Carrying capacity K={K}')
ax3.set_xlabel('Time t')
ax3.set_ylabel('Population P(t)')
ax3.set_title('Logistic Growth Model')
ax3.legend(loc='lower right')

# Plot 4: Harmonic oscillator position
ax4 = fig.add_subplot(gs[1, 1])
t_fine_harm = np.linspace(t0, tf, 1000)
ax4.plot(t_fine_harm, x_exact_harm(t_fine_harm), 'k-', linewidth=2, label='Exact')
ax4.plot(t_harm, y_harm[:, 0], 'b-', linewidth=1, alpha=0.7, label=f'Euler (h={h_harm})')
ax4.set_xlabel('Time t')
ax4.set_ylabel('Position x(t)')
ax4.set_title('Simple Harmonic Oscillator')
ax4.legend(loc='upper right')

# Plot 5: Phase space for harmonic oscillator
ax5 = fig.add_subplot(gs[2, 0])
theta = np.linspace(0, 2*np.pi, 100)
ax5.plot(np.cos(theta), -omega*np.sin(theta), 'k-', linewidth=2, label='Exact (circle)')
ax5.plot(y_harm[:, 0], y_harm[:, 1], 'b-', linewidth=1, alpha=0.7, label='Euler (spiral)')
ax5.plot(y_harm[0, 0], y_harm[0, 1], 'go', markersize=10, label='Start')
ax5.plot(y_harm[-1, 0], y_harm[-1, 1], 'r*', markersize=12, label='End')
ax5.set_xlabel('Position x')
ax5.set_ylabel('Velocity v')
ax5.set_title('Phase Space: Energy Drift in Euler Method')
ax5.legend(loc='upper right')
ax5.axis('equal')

# Plot 6: Energy evolution for harmonic oscillator
ax6 = fig.add_subplot(gs[2, 1])
# Total energy: E = 0.5*v^2 + 0.5*omega^2*x^2
energy_euler = 0.5 * y_harm[:, 1]**2 + 0.5 * omega**2 * y_harm[:, 0]**2
energy_exact = 0.5 * omega**2  # Initial energy (constant)
ax6.plot(t_harm, energy_euler, 'b-', linewidth=2, label='Euler method')
ax6.axhline(y=energy_exact, color='k', linestyle='--', linewidth=2, label='Exact (conserved)')
ax6.set_xlabel('Time t')
ax6.set_ylabel('Total Energy E')
ax6.set_title('Energy Conservation (Euler Fails)')
ax6.legend()

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved to plot.png")

## Summary and Discussion

### Key Observations

1. **First-Order Convergence**: The Euler method exhibits $O(h)$ global error, as verified by our convergence study.

2. **Step Size Sensitivity**: Larger step sizes lead to significant deviations from the exact solution, particularly for problems with rapid dynamics.

3. **Energy Drift**: For conservative systems like the harmonic oscillator, the explicit Euler method does not preserve energy. The numerical solution spirals outward in phase space, demonstrating artificial energy gain.

### Advantages

- Simple to understand and implement
- Computationally inexpensive per step
- Useful for educational purposes and quick prototyping

### Limitations

- Low accuracy requires small step sizes
- Poor stability properties for stiff equations
- Does not preserve geometric properties (symplecticity, energy conservation)

### Extensions

More accurate methods include:
- **Improved Euler (Heun's method)**: Second-order, $O(h^2)$
- **Runge-Kutta methods**: Higher-order accuracy (RK4 is $O(h^4)$)
- **Symplectic integrators**: For Hamiltonian systems
- **Implicit methods**: For stiff equations